In [1]:
#################################################
#created the 0/05/2018 14:27: by Alexis Blanchet#
#################################################
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import scipy.stats
import matplotlib
import plotly.offline as offline
import plotly.graph_objs as go
from sklearn.cluster import SpectralClustering
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from scipy import stats
from sklearn.metrics import log_loss
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
def plot_annomalies(df,anomalies):
    """
    prend en entrée le nombres d'indicateurs qui detectent une annomalie
    ainsi que le datafame contenant tous les features(on a effacé 3 lignes)
    renvoie un graph présentant les zones d'incertitudes quand a la présence
    d'un événement important dans la plage horaire
    en vert une montée d'audience, en orange une baisse d'audiance
    """
    anomalies = list(anomalies)
    l1 = find_index(anomalies,0)
    l2 = find_index(anomalies,-1)
    l3 = find_index(anomalies,1)

    x = df['t'].values
    t= [i for i in range(len(x))]
    x1 = [t[i] for i in l1]
    x2 = [t[i] for i in l2]
    x3 = [t[i] for i in l3]
    y1 = [x[i] for i in l1]
    y2 = [x[i] for i in l2]
    y3 = [x[i] for i in l3]

    
    trace1 = go.Scatter(
        x=x1,
        y=y1,
        mode = 'markers',
        name = 'regular',
    
    )
    trace2 = go.Scatter(
        x=x2,
        y=y2,
        mode = 'markers',
        name ='anormal loss',
    )
    trace3 = go.Scatter(
        x=x3,
        y=y3,
        mode = 'markers',
        name = 'anormal gain',
    )
    trace4 = go.Scatter(
        x=0,
        y=0,
        mode = 'markers',
        name = 'begin of programmes',
    )
        
    fig = tools.make_subplots(rows=4, cols=1, specs=[[{}], [{}], [{}], [{}]],
                              shared_xaxes=True, shared_yaxes=True,
                              vertical_spacing=0.001)
    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 1)
    fig.append_trace(trace3, 1, 1)
    fig.append_trace(trace4, 1, 1)

    fig['layout'].update(height=3000, width=2000, title='Annomalie detection')
    plot(fig, filename='data.html')

def find_index(l,v):
    res = []
    for i, j in enumerate(l):
        if(j == v):
            res.append(i)
    return res    

In [3]:
df = pd.read_csv('/home/alexis/Bureau/Stage/Time-series/data/processed/sfrdaily_20180430_0_192_0_cleandata-processed.csv')
y = pd.read_csv('/home/alexis/Bureau/Stage/Time-series/y_true2.csv')
df = df.drop(['label'],axis=1)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(1)
X_train = df.values
y_train = y['CP'][3:].values

In [4]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [5]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [6]:
# normalize the dataset
dataset = df
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
Y = y_train.reshape(-1, 1)
scalery = MinMaxScaler(feature_range=(0, 1))
Y = scalery.fit_transform(Y)

In [7]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
trainX, testX = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
trainY, testY = Y[0:train_size], Y[train_size:len(dataset)]
print(len(trainX), len(testX))

962 475


In [8]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
import tensorflow as tf


def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred*2), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 2 * precision * recall / (precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)




numpy.random.seed(99)
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, 27), dropout=0.4))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[f2_score])
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [58]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 0.36 RMSE
Test Score: 0.39 RMSE


# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))

plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [18]:
testPredict = list([1 if i[0]>0.15 else 0 for i in testPredict])
trainPredict = list([1 if i[0]>0.15 else 0 for i in trainPredict])

In [19]:
x = df['t'].values
t= [i for i in range(len(x))]


l1 = find_index(trainPredict,1)
l2 = find_index(testPredict,1)

x1 = [t[i] for i in l1]
x2 = [t[i+train_size] for i in l2]

y1 = [x[i] for i in l1]
y2 = [x[i+train_size] for i in l2]

l3 = find_index(y_train,1)
print(l3)
x3 = [t[i] for i in l3]
y3 = [x[i] for i in l3]


trace1 = go.Scatter(
        x= t,
        y= x,
        name = 'true',
    
)
trace2 = go.Scatter(
        x =x1,
        y=y1,
        mode = 'markers',
        name ='train',
)
trace3 = go.Scatter(
        x=x2,
        y= y2,
        mode = 'markers',
        name = 'test',
)
trace4 = go.Scatter(
        x=x3,
        y=y3,
        mode = 'markers',
        name = 'begin of programmes',
)
        
fig = tools.make_subplots(rows=4, cols=1, specs=[[{}], [{}], [{}], [{}]],
                              shared_xaxes=True, shared_yaxes=True,
                              vertical_spacing=0.001)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 1)
fig.append_trace(trace4, 1, 1)

fig['layout'].update(height=3000, width=2000, title='Annomalie detection')
plot(fig, filename='data.html')


[398, 409, 413, 428, 429, 441, 442, 454, 455, 456, 468, 469, 487, 494, 497, 518, 523, 529, 535, 568, 585, 588, 591, 595, 634, 642, 652, 656, 657, 676, 682, 711, 748, 757, 778, 784, 827, 828, 833, 849, 850, 859, 861, 869, 893, 896, 897, 915, 936, 960, 981, 1008, 1048, 1049, 1060, 1061, 1067, 1071, 1078, 1079, 1086, 1087, 1120, 1155, 1156, 1187, 1188, 1189, 1190, 1215, 1250, 1278, 1290, 1305, 1333, 1344, 1368, 1380]
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



'file:///home/alexis/Bureau/Stage/ML/data.html'

In [28]:
df2 = pd.read_csv('/home/alexis/Bureau/Stage/Time-series/data/processed/sfrdaily_20171215_0_192_0_cleandata-processed.csv')
df2 = df2.drop(['label'],axis=1).fillna(1)
X = scaler.fit_transform(df2)
X = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))

In [29]:
pred = model.predict(X)

In [31]:
v = [1 if x[0]>0.17 else 0 for x in pred]
x = df2['t'].values
t= [i for i in range(len(x))]


l1 = find_index(v,1)

x1 = [t[i] for i in l1]
y1 = [x[i] for i in l1]

trace1 = go.Scatter(
        x= t,
        y= x,
        name = 'true',
    
)
trace2 = go.Scatter(
        x =x1,
        y=y1,
        mode = 'markers',
        name ='train',
)
trace3 = go.Scatter(
        x=0,
        y= 0,
        mode = 'markers',
        name = 'test',
)
trace4 = go.Scatter(
        x=0,
        y=0,
        mode = 'markers',
        name = 'begin of programmes',
)
        
fig = tools.make_subplots(rows=4, cols=1, specs=[[{}], [{}], [{}], [{}]],
                              shared_xaxes=True, shared_yaxes=True,
                              vertical_spacing=0.001)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
#fig.append_trace(trace3, 1, 1)
#fig.append_trace(trace4, 1, 1)

fig['layout'].update(height=3000, width=2000, title='Annomalie detection')
plot(fig, filename='data.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



'file:///home/alexis/Bureau/Stage/ML/data.html'

In [32]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [33]:
model.save_weights("model.h5")